In [40]:
from music21 import *
from comp2 import *

In [41]:
melody_in: stream.Stream = parse_midi("example_inputs/eyeofthetiger.mid")
melody_in = melody_in.transpose(4)

print("Original melody:")
melody_in.show('midi');

Original melody:


In [42]:
print("Simple triads:")
simple_triads = get_triads(melody_in)

Simple triads:
['iii', 'ii', 'ii', 'ii', 'iii', 'iii', 'ii', 'iii', 'iii', 'ii', 'ii', 'ii', 'iii', 'iii', 'ii', 'iii', 'IV', 'I', 'IV', 'iii', 'IV', 'I', 'IV', 'vi', 'ii']


In [43]:
out_score = stream.Score()
out_score.insert(0, melody_in)
out_score.insert(0, simple_triads)

out_score.show('midi')

In [44]:
markov_table = generate_markov_table()
substituted_chords = substitute_chords(simple_triads, markov_table)
for pos_i in range(5):
    substituted_chords = substitute_chords(substituted_chords, markov_table)

out_score = stream.Score()
out_score.insert(0, melody_in)
out_score.insert(0, substituted_chords)

out_score.show('midi')

Total number of states: 2744

new chords:
['iv', 'ii2', 'IV7', 'v76', 'iii', 'iiisus4', 'iisus4', 'VI2', 'iii2', 'ii2', 'ii2', 'ii', 'iii7', 'iii2', 'VI76', 'iii6', 'IV7', 'Isus4', 'IV', 'VII2', 'IV76', 'I2', 'IV', 'ii2', 'ii7']

new chords:
['vii7', 'ii76', 'IVsus4', 'VII', 'i7', 'vi76', 'VII2', 'VI2', 'iii2', 'vi7', 'ii7', 'ii2', 'iii2', 'iii7', 'vi6', 'vii2', 'IV2', 'iv76', 'IV2', 'iv2', 'iii7', 'I2', 'IV7', 'VII2', 'ii']

new chords:
['viisus4', 'vii2', 'vii2', 'iii2', 'vi2', 'IV7', 'VIIsus4', 'VI7', 'iii2', 'vi7', 'ii2', 'v2', 'iii2', 'iii2', 'vi2', 'I2', 'IV6', 'ii2', 'IV2', 'iv', 'iii2', 'vi76', 'IV2', 'iv7', 'iisus4']

new chords:
['iii', 'vii7', 'vii2', 'iii7', 'iisus4', 'ii6', 'iiisus4', 'VI2', 'V2', 'vi2', 'ii7', 'i', 'iv2', 'vi2', 'visus4', 'ii2', 'IV2', 'v7', 'v2', 'iv2', 'iii2', 'IV7', 'IV2', 'iv7', 'i2']

new chords:
['iii2', 'vii2', 'vi2', 'iii', 'v2', 'ii7', 'vi2', 'VI7', 'V2', 'vi', 'ii2', 'iv2', 'iv2', 'vii7', 'ii', 'v2', 'v', 'v', 'v76', 'VI2', 'iii2', 'IV2', 'IV2',

In [45]:
import torch
from preprocess import *
from bigram import BigramModel

device = 'cuda' if torch.cuda.is_available() else 'cpu'

model_save_path = '/mnt/c/Users/jwest/Desktop/algocomps/comp2-markov/parkergpt'
checkpoint_f = torch.load(model_save_path + '_forward.pt', map_location=device)
checkpoint_b = torch.load(model_save_path + '_backward.pt', map_location=device)

model_f = BigramModel(
    device=device,
    block_size=checkpoint_f['block_size'],
    vocab_size=checkpoint_f['vocab_size'],
    n_embed=checkpoint_f['n_embed']
)

model_b = BigramModel(
    device=device,
    block_size=checkpoint_b['block_size'],
    vocab_size=checkpoint_b['vocab_size'],
    n_embed=checkpoint_b['n_embed']
)

# Load the saved weights
model_f.load_state_dict(checkpoint_b['model_state_dict'])
model_b.load_state_dict(checkpoint_b['model_state_dict'])

<All keys matched successfully>

In [46]:
gen_tokens_fwd = model_f.generate(
    idx=torch.zeros((1, 1), dtype=torch.long).to(device),
    max_length=100  # Generate sequence
)[0].tolist()

# Convert tokens to MIDI
gen_stream = sequence_to_score(tokens_to_sequence(gen_tokens_fwd))
gen_stream.show('midi')

In [ ]:
def splice_melody(melody: stream.Stream, past_context=6, future_context=8, max_tokens=20) -> stream.Stream:
    out = []
    pos = 0  # position in melody (in 24ths of a quarter note)
    pos_i = 0 # left boundary for current segment
    pos_j = 144 # right boundary
    idx = 0  # idx for forward model
    take = 'melody'
    
    while pos < melody.duration.quarterLength * 24:
        if take == 'melody':
            if pos_i > 144:
                # use the melody segments surrounding the slice to generate context
                # generate separate contexts for each models
                context_forward = make_tokens(score_to_sequence(out[-past_context:]))

                # use element index to get future context for backward model
                context_backward = make_tokens(score_to_sequence(
                    melody.flatten().notesAndRests.getElementsByOffset(
                        (pos + 144) / 24,
                        ((pos + 144) / 24) + future_context
                    )[::-1] # reverse the context
                ))
                # if backward context is empty, pad with rests
                if len(context_backward) == 0:
                    final_rests = []
                    for i in range(future_context):
                        final_rests.append(note.Rest(quarterLength=1))
                    context_backward = make_tokens(score_to_sequence(final_rests))

                print(f"\nforward context: {context_forward}")
                print(f"backward context: {context_backward}")

                gen_tokens_fwd = sequence_to_score(tokens_to_sequence(model_f.generate(
                    idx=torch.tensor(context_forward, dtype=torch.long).unsqueeze(0).to(device),
                    max_length=max_tokens
                )[0].tolist())).flatten().notesAndRests
                print(f"forward gen: {list(gen_tokens_fwd)}")

                gen_tokens_back = sequence_to_score(tokens_to_sequence(model_b.generate(
                    idx=torch.tensor(context_backward, dtype=torch.long).unsqueeze(0).to(device),
                    max_length=max_tokens
                )[0].tolist())[::-1]).flatten().notesAndRests
                print(f"backward gen: {list(gen_tokens_back)}")

                pos_i = 0
                pos_j = 144
                idx = 0
                take = 'model'
                continue

            # take next element from melody
            elem_fwd = melody.flatten().getElementAtOrBefore(pos / 24)
            if elem_fwd is None:
                break
            out.append(elem_fwd)
            pos += int(elem_fwd.quarterLength * 24)
            pos_i += int(elem_fwd.quarterLength * 24)
            
        else:
            if pos_i < pos_j:
                pos += 144
                take = 'melody'
                continue
                
            # Get elements by index
            # for pos, pool in [(pos_i, gen_tokens_fwd), (pos_j, gen_tokens_back)]:

            elem_fwd = gen_tokens_fwd[idx]
            
            # Create a copy of the element to prevent ID conflicts
            if isinstance(elem_fwd, note.Note):
                elem_fwd = note.Note(pitch=elem_fwd.pitch, quarterLength=elem_fwd.quarterLength)
            elif isinstance(elem_fwd, note.Rest):
                elem_fwd = note.Rest(quarterLength=elem_fwd.quarterLength)
            
            elem_fwd.offset = pos_i / 24 # Set offset
            pos_i += int(elem_fwd.quarterLength * 24) # Then increment position
            out.append(elem_fwd) # Then append
            print(f"forward gen: appending {elem_fwd}")

            elem_back = gen_tokens_back[idx]
            if isinstance(elem_back, note.Note):
                elem_back = note.Note(pitch=elem_back.pitch, quarterLength=elem_back.quarterLength)
            elif isinstance(elem_back, note.Rest):
                elem_back = note.Rest(quarterLength=elem_back.quarterLength)
            
            pos_j -= int(elem_back.quarterLength * 24) # Decrement position
            elem_back.offset = pos_j / 24 # Then set offset
            out.append(elem_back) # Then append
            print(f"backward gen: appending {elem_back}")

            idx += 1  # Increment index for both streams
            if idx >= max_tokens:
                pos += 0
                take = 'melody'
                continue
                    
    return stream.Stream(out)


In [73]:
res_melody = splice_melody(melody_in, past_context=6, future_context=3, max_tokens=20)

out_score = stream.Score()
out_score.insert(0, res_melody)
out_score.insert(0, substituted_chords)

out_score.show('midi')


forward context: [1843, 49, 37, 1787, 1737, 1837, 1793]
backward context: [49, 49, 1749, 31]
forward gen: [<music21.note.Note B>, <music21.note.Rest quarter>, <music21.note.Rest eighth>, <music21.note.Note A>, <music21.note.Note G>, <music21.note.Note B>, <music21.note.Note A>, <music21.note.Rest dotted-eighth>, <music21.note.Note C#>, <music21.note.Note E>, <music21.note.Note E->, <music21.note.Note D>, <music21.note.Note F#>, <music21.note.Note A>, <music21.note.Note C>, <music21.note.Rest eighth>, <music21.note.Rest quarter>, <music21.note.Note B>, <music21.note.Note F>, <music21.note.Note G#>, <music21.note.Note D>, <music21.note.Note F>, <music21.note.Note A>, <music21.note.Note A>, <music21.note.Note B>, <music21.note.Note B->, <music21.note.Note F#>]
backward gen: [<music21.note.Note E->, <music21.note.Note B->, <music21.note.Note A>, <music21.note.Note B->, <music21.note.Note D>, <music21.note.Note G>, <music21.note.Note B->, <music21.note.Note C>, <music21.note.Note B>, <musi

In [74]:
out_score.write('midi', 'tiger_out.mid')

'tiger_out.mid'